In [1]:
import torch
import torch.utils.data as torch_split
import numpy as np
import dataset
import test
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
from monai.losses import DiceLoss
from monai.networks.nets import UNet
from monai.data import DataLoader
import sys
sys.path.insert(1, 'H:/Projects/Kaggle/CZII-CryoET-Object-Identification/preprocessing')


In [2]:
path = "H:/Projects/Kaggle/CZII-CryoET-Object-Identification/datasets/3D/dim96-no-corner"
data = dataset.UNetDataset(path=path)

tv_split = 0.8
trn = int(len(data) * tv_split)
val = len(data) - trn

train_dataset, val_dataset = torch_split.random_split(data, [trn, val])
# train_dataset = dataset.UNetDataset(path=path, train=True)
# val_dataset = dataset.UNetDataset(path=path, val=True)

In [3]:
import optuna
import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from monai.losses import DiceLoss
from monai.networks.nets import UNet

# Define the objective function for Optuna
def objective(trial):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Hyperparameters to optimize
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-3)
    batch_size = trial.suggest_categorical("batch_size", [4, 8, 16])
    dropout = trial.suggest_uniform("dropout", 0.0, 0.5)
    regularization_type = trial.suggest_categorical("regularization_type", ["none", "L1", "L2"])

    # Suggest regularization strength only if regularization is used
    if regularization_type != "none":
        regularization_strength = trial.suggest_loguniform("regularization_strength", 1e-5, 1e-3)
    else:
        regularization_strength = 0  # No regularization if "none"

    print(f"LR: {lr}\tbatch size: {batch_size}\tdropout: {dropout}\treg type: {regularization_type}\treg str: {regularization_strength}")
    
    # Model initialization
    model = UNet(
        spatial_dims=3,
        in_channels=1,
        out_channels=7,
        channels=(64, 128, 256, 512),
        strides=(2, 2, 2, 2),
        num_res_units=2,
        dropout=dropout,
    ).to(device)

    # Loss function and optimizer
    criterion = DiceLoss(to_onehot_y=True, softmax=True).to(device)
    optimizer = Adam(model.parameters(), lr=lr)

    # DataLoader (update batch size)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=dataset.collate_fn, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, collate_fn=dataset.collate_fn, shuffle=False, num_workers=4)

    # Regularization setup
    def add_regularization_loss(model, regularization_type, regularization_strength):
        reg_loss = 0
        if regularization_type == "L1":
            for param in model.parameters():
                reg_loss += torch.sum(torch.abs(param))
        elif regularization_type == "L2":
            for param in model.parameters():
                reg_loss += torch.sum(param ** 2)
        return regularization_strength * reg_loss

    # Early stopping parameters
    patience = 2
    best_val_loss = float('inf')
    epochs_without_improvement = 0
    elapsed = 0

    # Training loop
    num_epochs = 15  # Shorten for quicker optimization
    for epoch in range(num_epochs):
        model.train()
        for batch in train_loader:
            elapsed += 1
            
            inputs, targets = batch['src'].float().to(device), batch['tgt'].long().to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            # Add regularization loss if applicable
            if regularization_type != "none":
                reg_loss = add_regularization_loss(model, regularization_type, regularization_strength)
                loss += reg_loss

            loss.backward()
            optimizer.step()

        # Validation loop
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                inputs, targets = batch['src'].float().to(device), batch['tgt'].long().to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item()

        val_loss /= len(val_loader)

        # Early stopping check
        if val_loss <= best_val_loss * 0.99:
            best_val_loss = val_loss
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= patience:
                print(f"Early stopping triggered at epoch {epoch + 1}")
                break
    print(f"Validation loss: {best_val_loss} after {elapsed} epochs")
    return best_val_loss

# Run the Optuna optimization
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=25)

# Print the best parameters
print("Best hyperparameters:", study.best_params)

[I 2024-12-22 11:30:42,695] A new study created in memory with name: no-name-6e8b94f9-f607-498b-a3fb-39dc953982af
C:\Users\hmhor\AppData\Local\Temp\ipykernel_4860\361824959.py:13: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform("lr", 1e-5, 1e-3)
C:\Users\hmhor\AppData\Local\Temp\ipykernel_4860\361824959.py:15: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout = trial.suggest_uniform("dropout", 0.0, 0.5)
c:\Users\hmhor\AppData\Local\Programs\Python\Python310\lib\site-packages\monai\networks\nets\unet.py:130: UserWarning: `len(strides) > len(channels) - 1`, the last 1 values of strides will not be used.
  warnings.warn(f"`len(strides) > len(channels) - 1`, t

LR: 0.0007690852832005317	batch size: 16	dropout: 0.20627530165636548	reg type: none	reg str: 0


[I 2024-12-22 11:37:38,925] Trial 0 finished with value: 0.7081862275417035 and parameters: {'lr': 0.0007690852832005317, 'batch_size': 16, 'dropout': 0.20627530165636548, 'regularization_type': 'none'}. Best is trial 0 with value: 0.7081862275417035.


Early stopping triggered at epoch 8
Validation loss: 0.7081862275417035 after 400 epochs
LR: 8.02200311056887e-05	batch size: 8	dropout: 0.38585196148834683	reg type: L1	reg str: 0.000561752035750931


C:\Users\hmhor\AppData\Local\Temp\ipykernel_4860\361824959.py:20: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  regularization_strength = trial.suggest_loguniform("regularization_strength", 1e-5, 1e-3)
[I 2024-12-22 11:43:47,207] Trial 1 finished with value: 0.8221715784072876 and parameters: {'lr': 8.02200311056887e-05, 'batch_size': 8, 'dropout': 0.38585196148834683, 'regularization_type': 'L1', 'regularization_strength': 0.000561752035750931}. Best is trial 0 with value: 0.7081862275417035.


...

Early stopping triggered at epoch 15
Validation loss: 0.671155903339386 after 1500 epochs
LR: 0.00018330211028207545	batch size: 8	dropout: 0.09018376947727601	reg type: L1	reg str: 0.0002595641269967224


[I 2024-12-22 15:15:31,993] Trial 23 finished with value: 0.7843618059158325 and parameters: {'lr': 0.00018330211028207545, 'batch_size': 8, 'dropout': 0.09018376947727601, 'regularization_type': 'L1', 'regularization_strength': 0.0002595641269967224}. Best is trial 15 with value: 0.6498052048683166.


Early stopping triggered at epoch 9
Validation loss: 0.7843618059158325 after 900 epochs
LR: 0.00032731036287932553	batch size: 8	dropout: 0.24536040493307815	reg type: L2	reg str: 2.0610968568687057e-05


[I 2024-12-22 15:28:26,822] Trial 24 finished with value: 0.6468523573875428 and parameters: {'lr': 0.00032731036287932553, 'batch_size': 8, 'dropout': 0.24536040493307815, 'regularization_type': 'L2', 'regularization_strength': 2.0610968568687057e-05}. Best is trial 24 with value: 0.6468523573875428.


Validation loss: 0.6468523573875428 after 1500 epochs
Best hyperparameters: {'lr': 0.00032731036287932553, 'batch_size': 8, 'dropout': 0.24536040493307815, 'regularization_type': 'L2', 'regularization_strength': 2.0610968568687057e-05}


In [ ]:
import numpy as np
from ipywidgets import interact
import augment
import load
import matplotlib.pyplot as plt


root = load.get_root()

picks = load.get_picks_dict(root)

vol, coords, scales = load.get_run_volume_picks(root, level=0)

mask = load.get_picks_mask(vol.shape, picks, coords, int(scales[0]))

In [26]:
params = augment.aug_params
params['patch_size'] = (96,96,96)
params['final_size'] = (96,96,96)
params['flip_prob'] - 0.0
params['rot_prob'] = 0.0
params['rot_range'] = 0.0



samples = augment.random_augmentation(vol, mask, num_samples=1, aug_params=params)

model.eval()

inp = np.array(samples[0]["source"].unsqueeze(0).unsqueeze(0))
inp = torch.from_numpy(inp).to(device)
pred_mask = model(inp)

src = samples[0]['source']
tgt = samples[0]['target']  # Mask with interest points (non-zero values)

pred_tgt = pred_mask.squeeze().cpu().detach()


pred_tgt = torch.argmax(pred_tgt, dim = 0).numpy()


print(f'# Particles Types Represented: {len(np.unique(tgt)) - 1}')
print(f'# Particles Types Predicted: {len(np.unique(pred_tgt)) - 1}')


def plot_cross_section(i):
    plot_vol = tgt
    plot_mask = pred_tgt
    
    plt.figure(figsize=(15, 5))
    alpha = 0.3

    # Slice at x-coordinate
    plt.subplot(131)
    plt.imshow(plot_vol[i, :, :], cmap="viridis")
    plt.imshow(plot_mask[i, :, :], cmap="Reds", alpha=alpha)  # Overlay mask with transparency
    plt.title(f'Slice at x={i}')

    # Slice at y-coordinate
    plt.subplot(132)
    plt.imshow(plot_vol[:, i, :], cmap="viridis")
    plt.imshow(plot_mask[:, i, :], cmap="Reds", alpha=alpha)
    plt.title(f'Slice at y={i}')

    # Slice at z-coordinate
    plt.subplot(133)
    plt.imshow(plot_vol[:, :, i], cmap="viridis")
    plt.imshow(plot_mask[:, :, i], cmap="Reds", alpha=alpha)
    plt.title(f'Slice at z={i}')

    plt.show()

# Interactive Slider for scrolling through slices
interact(plot_cross_section, i=(0, tgt.shape[0] - 1))

# Particles Types Represented: 4
# Particles Types Predicted: 6


interactive(children=(IntSlider(value=47, description='i', max=95), Output()), _dom_classes=('widget-interact'…

<function __main__.plot_cross_section(i)>